# Dispositivos de captura

Para hacer experimentos de visión artificial nos interesa acceder cómodamente a cualquier fuente de imágenes, almacenada localmente o disponible en un servidor remoto. Por ejemplo:

- archivos de imagen (jpg, png, tif, bmp, etc.) 

- archivos de video (avi, mp4, etc.)

- secuencias "en vivo" tomadas de una webcam, cámara ip, o smartphone.

## autoStream

OpenCV (y otros paquetes como scikit-image, pillow, etc.) permiten leer imágenes estáticas sin problemas. Pero no todas las versiones precompiladas de OpenCV incluyen un soporte de captura de vídeo suficientemente general, por lo que utilizaremos un convertidor a formato [mjpeg](https://en.wikipedia.org/wiki/Motion_JPEG), que puede leerse fácilmente con una función auxiliar disponible en el paquete umucv.

La función `mkStream(size,dev)` crea un generador de imágenes:

    stream = mkStream(size,dev)

    for frame in stream:        
        procesa frame

Para capturar el teclado podemos usar directamente `cv.waitKey` pocesando los códigos como deseemos.

Muchas veces es más cómodo usar `autoStream`, que permite hacer pausas (espacio) salvar frames (tecla s), salir (ESC), y admite opciones en la línea de órdenes para elegir el dispositivo de captura (`--dev`) y la resolución (`--size`).

    for key, frame in autoStream():
        
        if key = ord('c'):
            procesa tecla
        
        procesa frame


Por omisión el dispositivo es la cámara /dev/video0 y el tamaño 640x480. Para cambiar de webcam y tamaño hacemos:

    ./programa.py --dev=1 --size=320x240
    
La cámara del raspberry pi se especifica así:

    ./programa.py --dev=picam

Podemos procesar una lista de imágenes tomadas de un directorio:

    ./programa.py --dev=glob:../images/ccorr/scenes/*.png

Otra forma de hacer lo mismo, pero visualizando las imágenes disponibles en una ventana, avanzando de una en una con las teclas del ratón:

    ./programa.py --dev=dir:../images/ccorr/scenes/*.png

Se admiten archivos de vídeo en formato mjpeg locales:

    ./programa.py --dev=file:../images/rot4.mjpg
    

Si el vídeo está en otro formato lo convertimos con la utilidad `ffmpeg` (que puede instalarse mediante conda):

    ffmpeg -i video.avi -c:v mjpeg -q:v 3 -huffman optimal -an video.mjpg
    
También se admiten *streams* mjpeg remotos, generados por [cámaras online](https://en.wikipedia.org/wiki/IP_camera#Video_standards). Por ejemplo, aquí vemos una playa:

    ./programa.py --dev=http://213.4.39.225:81/mjpg/video.mjpg

La *app* "IP Webcam" incluye un servidor mjpeg en los smartphones, que permite utilzarlos como fuente de imágenes para los ejercicios.

    ./programa.py --dev=http://155.54.X.Y:8080/video


## Servidores

Hemos incluido dos utilidades para crear "streams" en formato mjpeg:

- ./`mjpegserver.py`: crea un stream mjpeg a partir cualquier dispositivo de los anteriores. Por ejemplo:

        ./mjpegserver.py --dev=picam --size=320x240 --quality=50
        
    Este stream se captura con
    
        ./programa.py --dev=http://<IPDELRASPBERY>:8087/cam.mjpg
        
- ./`vlcmjpeg.sh`: es un script de bash para llamar a `vlc` con cualquier fuente de video y generar un stream mjpeg:

        ./vlcmjpeg.sh https://www.youtube.com/watch?v=aBr2kKAHN6M
        
    (Es conveniente abrir el vídeo primero con vlc normal para comprobar que se lee bien, y en su caso aceptar certificados.)
    
    Este stream se captura con
    
        ./programa.py --dev=http://<IPDELSERVIDOR>:8090
    
    Podemos transmitir la pantalla de nuestro ordenador:
        
        ./vlcmjpeg.sh 'screen:// :screen-fps=10 :screen-width=700 :screen-height=500 :screen-top=300'
        
    O la webcam:
    
        ./vlcmjpeg.sh 'v4l2://'    
    
    En, general, podemos emitir cualquier fuente de video admitida por `vlc`. Estas secuencias de imágenes se pueden capturar fácilmente con las funciones `mkStream()` y derivadas mostradas anteriormente.

## Control de la webcam

En la consola de linux usamos **v4l2-ctl**. Los controles disponibles en cada cámara se consultan con:

    v4l2-ctl -l

Activar o desactivar la eliminación de oscilaciones producidas por la luz eléctrica:

        v4l2-ctl -d /dev/video0 -c power_line_frequency=1
        v4l2-ctl -d /dev/video0 -c power_line_frequency=0

Fijar el nivel de exposición:

    v4l2-ctl -d /dev/video0 -c exposure_auto=1 -c exposure_absolute=100
    v4l2-ctl -d /dev/video0 -c exposure_auto=1 -c exposure_absolute=1000

Exposición automática:

    v4l2-ctl -d /dev/video0 -c exposure_auto=3

Enfoque fijo:

    v4l2-ctl -d /dev/video0 -c focus=255
    v4l2-ctl -d /dev/video0 -c focus=0

La aplicación *guvcview* permite modificar los parámetros con un interfaz gráfico. 

El reproductor multimedia VLC (disponible en Linux y Windows) también lo permite.